In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
from tactile_learning.datasets.preprocess import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [3]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, 'cam_0_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [12]:

def dump_states(root, tactile_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [13]:
import glob
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_3',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_2',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_22',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_21',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_12',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_9',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_20',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_25',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_16',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_7',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_24',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_5',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_19',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_4',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_27',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_17'
# ]
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training'
roots = glob.glob(f'{data_path}/demonstration_*')

retrieval_fps = 2
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices, view_num=1)


tactile_indices: [0, 52, 102, 149, 197, 246, 296, 344, 393, 442, 491, 541, 589, 639, 688, 738, 785, 834, 882, 932, 980, 1032, 1078, 1128, 1176, 1227], image_indices: [69, 83, 98, 112, 126, 140, 155, 170, 185, 200, 214, 229, 244, 259, 274, 289, 304, 318, 333, 348, 361, 376, 388, 400, 412, 426]


100%|█████████████████████| 26/26 [00:41<00:00,  1.60s/it]


tactile_indices: [0, 51, 99, 150, 197, 247, 294, 345, 392, 442, 491, 541, 589, 638, 687], image_indices: [71, 85, 99, 114, 128, 143, 157, 172, 185, 200, 215, 230, 245, 260, 275]


100%|█████████████████████| 15/15 [00:18<00:00,  1.23s/it]


tactile_indices: [0, 51, 99, 149, 197, 248, 295, 345, 394, 442, 490, 540, 590, 636, 687, 735, 784, 832, 880, 929, 977, 1028, 1078], image_indices: [67, 81, 96, 110, 124, 139, 153, 167, 180, 194, 208, 223, 237, 251, 266, 280, 294, 307, 321, 335, 348, 362, 376]


...

100%|█████████████████████| 21/21 [00:25<00:00,  1.23s/it]


tactile_indices: [0, 52, 99, 150, 198, 247, 295, 345, 394, 443, 491, 542, 589, 639, 687, 736], image_indices: [71, 85, 100, 115, 130, 145, 160, 175, 189, 204, 218, 233, 247, 262, 277, 291]


100%|█████████████████████| 16/16 [00:20<00:00,  1.28s/it]


tactile_indices: [0, 51, 101, 149, 199, 248, 297, 346, 394, 443, 493, 542, 591, 639, 690, 738, 788, 836, 889, 934, 983, 1032, 1080, 1129, 1177, 1228, 1278], image_indices: [70, 85, 98, 113, 128, 143, 157, 172, 186, 201, 216, 230, 245, 260, 275, 290, 305, 319, 334, 348, 363, 377, 392, 407, 421, 436, 451]


100%|█████████████████████| 27/27 [00:34<00:00,  1.27s/it]


In [14]:
print(roots)

['/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_13', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_26', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_23', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_10', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_22', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_21', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_12', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_20', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_25', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_24', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_19', '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_27']


In [15]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, 'visualization.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_13


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_26


[libx264 @ 0x557b53cebf40] using SAR=1/1
[libx264 @ 0x557b53cebf40] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x557b53cebf40] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x557b53cebf40] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonst

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_23


...

[libx264 @ 0x564219e13240] using SAR=1/1
[libx264 @ 0x564219e13240] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x564219e13240] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x564219e13240] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonst

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_19


[libx264 @ 0x559df3e9afc0] using SAR=1/1
[libx264 @ 0x559df3e9afc0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x559df3e9afc0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x559df3e9afc0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonst

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonstration_27


[libx264 @ 0x55b48f79f180] using SAR=1/1
[libx264 @ 0x55b48f79f180] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55b48f79f180] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x55b48f79f180] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/gamepad/training/demonst

In [16]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
